<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Writing-An-Parser-In-Nim" data-toc-modified-id="Writing-An-Parser-In-Nim-1">Writing An Parser In Nim</a></span></li><li><span><a href="#目次" data-toc-modified-id="目次-2">目次</a></span></li><li><span><a href="#概要" data-toc-modified-id="概要-3">概要</a></span></li><li><span><a href="#What-is-Parser?" data-toc-modified-id="What-is-Parser?-4">What is Parser?</a></span></li><li><span><a href="#demo" data-toc-modified-id="demo-5">demo</a></span></li><li><span><a href="#Example" data-toc-modified-id="Example-6">Example</a></span><ul class="toc-item"><li><span><a href="#Lexer" data-toc-modified-id="Lexer-6.1">Lexer</a></span></li><li><span><a href="#Struct" data-toc-modified-id="Struct-6.2">Struct</a></span></li><li><span><a href="#AST" data-toc-modified-id="AST-6.3">AST</a></span></li></ul></li><li><span><a href="#Pratt-Parser" data-toc-modified-id="Pratt-Parser-7">Pratt Parser</a></span><ul class="toc-item"><li><span><a href="#仕組み" data-toc-modified-id="仕組み-7.1">仕組み</a></span><ul class="toc-item"><li><span><a href="#完成品" data-toc-modified-id="完成品-7.1.1">完成品</a></span></li><li><span><a href="#2-*-3を作る" data-toc-modified-id="2-*-3を作る-7.1.2">2 * 3を作る</a></span><ul class="toc-item"><li><span><a href="#1" data-toc-modified-id="1-7.1.2.1">1</a></span></li><li><span><a href="#2" data-toc-modified-id="2-7.1.2.2">2</a></span></li><li><span><a href="#3" data-toc-modified-id="3-7.1.2.3">3</a></span></li></ul></li></ul></li><li><span><a href="#/-4の部分を作る" data-toc-modified-id="/-4の部分を作る-7.2">/ 4の部分を作る</a></span></li></ul></li></ul></div>

# Writing An Parser In Nim

# Table of Contents

1. 

# 目次

1. 概要
2. parserとは
3. demo
3. parserの種類
4. Pratt構文解析器
5. visualizerとか
6. 苦労したこと
6. 所感

# 概要

- Goで作るインタプリタの２章が終わった
- Lexerの続き
![](https://www.oreilly.co.jp/books/images/picture_large978-4-87311-822-2.jpeg)

# What is Parser?


# Parserとは何か

- Token列からASTを作るやつ
- ASTは、プログラムの抽象的な構文木のこと

# demo

`$ nim c -r src/repl/repl.nim`

# Example

![](https://github.com/mrsekut/slides/blob/master/img/WritingParserInNim/inputLetStatementCode.png?raw=true)

## Lexer

```
[Type = "LET", Literal = "let"]
[Type = "IDENT", Literal = "hoge"]
[Type = "=", Literal = "="]
[Type = "INT", Literal = "1"]
[Type = "+", Literal = "+"]
[Type = "INT", Literal = "2"]
[Type = "*", Literal = "*"]
[Type = "INT", Literal = "3"]
[Type = "/", Literal = "/"]
[Type = "INT", Literal = "4"]
[Type = "+", Literal = "+"]
[Type = "INT", Literal = "5"]
[Type = "-", Literal = "-"]
[Type = "INT", Literal = "6"]
[Type = ";", Literal = ";"]
```

## Struct
```
Token: {
    Type: 'LET',
    Literal: 'let'
},
Let: {
    Token: {
        Type: 'IDENT',
        Literal: 'hoge'
    },
    IdentValue: 'hoge'
},
LetValue: {
    Left: {
        Left: {
            Token: {
                Type: 'INT',
                Literal: '1'
            },
            IntValue: 1
        },
        Operator: '+',
        Right: {
            Left: {
                Left: {
                    Token: {
                        Type: 'INT',
                        Literal: '2'
                    },
                    IntValue: 2
                },
                Operator: '*',
                Right: {
                    Token: {
                        Type: 'INT',
                        Literal: '3'
                    },
                    IntValue: 3
                }
            },
            Operator: '/',
            Right: {
                Token: {
                    Type: 'INT',
                    Literal: '4'
                },
                IntValue: 4
            }
        }
    },
    Operator: '+',
    InRight: {
        Left: {
            Token: {
                Type: 'INT',
                Literal: '5'
            },
            IntValue: 5
        },
        Operator: '*',
        Right: {
            Token: {
                Type: 'INT',
                Literal: '6'
            },
            IntValue: 6
        }
    }
}

```

## AST

雑なAST  
![](https://github.com/mrsekut/slides/blob/master/img/WritingParserInNim/ast.png?raw=true)

# Pratt Parser

- Vaughan Pratt, 1973
- he says 
    - very simple to understand
    - trivial to implement
    - easy to use
    - extremely efficient in practice if not in theory
    
    
論文→https://tdop.github.io/
jsを用いた解説http://crockford.com/javascript/tdop/tdop.html ←jslintを作った人

## Types of parsers

- Top-down parsing
    - recurcive-descent parser ← this!
    - LL parser
    - packrat parser
- Bottom-up parsing
    - LR parser
        - Simple LR parser
        - LALR parser
        - CLR parser
        - GLR parser
     - Earley parser
     - CYK parser






https://en.wikipedia.org/wiki/Parsing

## 仕組み

- 前提
    - 式と文
    
### Expression

- 値を生成する
- ex.
    - 42, hoge


### Statement

- 値を生成しない
- ex.
    - let hoge = foo
    - return 42
    - if else

### 完成品

"2 * 3 / 4"

![](https://github.com/mrsekut/slides/blob/master/img/WritingParserInNim/ast2.png?raw=true)


![](https://github.com/mrsekut/slides/blob/master/img/WritingParserInNim/struct.png?raw=true)

### 2 * 3を作る

#### 1

- parseStatement

![](https://github.com/mrsekut/slides/blob/master/img/WritingParserInNim/1.png?raw=true)

![](https://github.com/mrsekut/slides/blob/master/img/WritingParserInNim/parseStatementCode-2.png?raw=true)



#### 2



![](https://github.com/mrsekut/slides/blob/master/img/WritingParserInNim/2.png?raw=true)

![](https://github.com/mrsekut/slides/blob/master/img/WritingParserInNim/parseExpressionCode-INT.png?raw=true)


parseExpressionStatement()の中でparseExpression()が呼び出されます。

- 図
- こｋ−土bmbm

#### 3
- whileにはいった

![](https://github.com/mrsekut/slides/blob/master/img/WritingParserInNim/3.png?raw=true)

![](https://github.com/mrsekut/slides/blob/master/img/WritingParserInNim/parseExpressionCode-PLUS.png?raw=true)


4
![](https://github.com/mrsekut/slides/blob/master/img/WritingParserInNim/4.png?raw=true)


![](https://github.com/mrsekut/slides/blob/master/img/WritingParserInNim/parseInfixExpression-2.png?raw=true)

5

- 再びparseExpressionが呼び出された
- ここではwhileループに入らない(ホンマに？)

![](https://github.com/mrsekut/slides/blob/master/img/WritingParserInNim/5.png?raw=true)

![](https://github.com/mrsekut/slides/blob/master/img/WritingParserInNim/parseExpressionCode-INT-2.png?raw=true)
|

6
- parseInfixExpressionの値が返ってきたので、whileの一番上に戻る

![](https://github.com/mrsekut/slides/blob/master/img/WritingParserInNim/6.png?raw=true)

![](https://github.com/mrsekut/slides/blob/master/img/WritingParserInNim/parseExpressionCode-while.png?raw=true)


### 優先順位を判定する

while文に入るかどうか



```
Lowest < Equals # true
Call < Lowest # false
```


![](https://github.com/mrsekut/slides/blob/master/img/WritingParserInNim/PrecedenceCode.png?raw=true)




"+" == "-"
![](https://github.com/mrsekut/slides/blob/master/img/WritingParserInNim/tokenToPrecedenceCode.png?raw=true)


## / 4の部分を作る

7
![](https://github.com/mrsekut/slides/blob/master/img/WritingParserInNim/7.png?raw=true)

![](https://github.com/mrsekut/slides/blob/master/img/WritingParserInNim/parseExpressionCode-SLASH.png?raw=true)


![](https://github.com/mrsekut/slides/blob/master/img/WritingParserInNim/parseInfixExpression-Operator.png?raw=true)

8
![](https://github.com/mrsekut/slides/blob/master/img/WritingParserInNim/8.png?raw=true)

![](https://github.com/mrsekut/slides/blob/master/img/WritingParserInNim/parseInfixExpression-right.png?raw=true)

9
![](https://github.com/mrsekut/slides/blob/master/img/WritingParserInNim/9.png?raw=true)

![](https://github.com/mrsekut/slides/blob/master/img/WritingParserInNim/parseInfixExpression.png?raw=true)